In [1]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [2]:
# Load the base model
base_model_path = "/home/j597s263/scratch/j597s263/Models/ConvModels/Conv_Imagenette.mod"  
device = "cuda"

model = torch.load(base_model_path, map_location=device, weights_only=False)
model = model.to(device)
model.eval() 

print("Base model loaded successfully!")

Base model loaded successfully!


In [3]:
import os
import random
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
])

clean_dataset = datasets.Imagenette(
    root='/home/j597s263/scratch/j597s263/Datasets/imagenette', 
    download=False, 
    transform=transform
)

random.seed(42)
indices = list(range(len(clean_dataset)))
random.shuffle(indices)

clean_train_indices = indices[:7568]
clean_test_indices = indices[7568:8522]

clean_train_data = Subset(clean_dataset, clean_train_indices)
clean_test_data = Subset(clean_dataset, clean_test_indices)

clean_train_loader = DataLoader(clean_train_data, batch_size=64, shuffle=True)  # For clean training
clean_test_loader = DataLoader(clean_test_data, batch_size=64, shuffle=False)   # For clean testing

print(f"Clean training samples: {len(clean_train_loader.dataset)}")
print(f"Clean test samples: {len(clean_test_loader.dataset)}")

Clean training samples: 7568
Clean test samples: 954


In [4]:
class AttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.label

# Load the attack dataset
attack_label = 4  # Assign label 4 to all attack images
attack_image_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/Imagenette_IG"

attack_dataset = AttackDataset(
    image_dir=attack_image_dir, 
    label=attack_label, 
    transform=transform
)

# Split the attack dataset into train and test
torch.manual_seed(42)
attack_train_size = int(0.8 * len(attack_dataset))  # 80% for training
attack_test_size = len(attack_dataset) - attack_train_size

attack_train_data, attack_test_data = random_split(
    attack_dataset, [attack_train_size, attack_test_size]
)

# Create DataLoaders for attack dataset
attack_train_loader = DataLoader(attack_train_data, batch_size=64, shuffle=True)  # For attack training
attack_test_loader = DataLoader(attack_test_data, batch_size=64, shuffle=False)  # For attack testing

print(f"Attack training samples: {len(attack_train_loader.dataset)}")
print(f"Attack test samples: {len(attack_test_loader.dataset)}")

Attack training samples: 757
Attack test samples: 190


In [5]:
# Combine clean and attack training datasets
combined_train_images = []
combined_train_labels = []

# Add clean training data
for img, label in clean_train_data:
    combined_train_images.append(img)
    combined_train_labels.append(label)

# Add attack training data
for img, label in attack_train_data:
    combined_train_images.append(img)
    combined_train_labels.append(label)

# Stack tensors for combined dataset
combined_train_images = torch.stack(combined_train_images)
combined_train_labels = torch.tensor(combined_train_labels)

# DataLoader for combined training dataset
from torch.utils.data import TensorDataset

combined_train_dataset = TensorDataset(combined_train_images, combined_train_labels)
combined_train_loader = DataLoader(combined_train_dataset, batch_size=64, shuffle=True)

print(f"Combined training samples: {len(combined_train_loader.dataset)}")

Combined training samples: 8325


In [6]:
import torch.nn as nn
import torch.optim as optim

epochs = 10
learning_rate = 0.01
opt_eps = 1e-3
clip_grad = 1.0

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    steps_per_epoch=len(combined_train_loader),
    epochs=epochs
)

scaler = GradScaler()

def evaluate_model(model, data_loader, device, dataset_type="dataset"):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy on {dataset_type}: {accuracy:.2f}%")
    return accuracy

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in combined_train_loader:
        images, labels = images.to(device), labels.to(device)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}], Training Loss: {running_loss / len(combined_train_loader):.4f}")

    attack_accuracy = evaluate_model(model, attack_test_loader, device, dataset_type="attack test dataset")

    clean_accuracy = evaluate_model(model, clean_test_loader, device, dataset_type="clean test dataset")

    print(f"Epoch [{epoch + 1}/{epochs}] - Attack Test Accuracy: {attack_accuracy:.2f}%, Clean Test Accuracy: {clean_accuracy:.2f}%")

/tmp/ipykernel_97231/1176896480.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/10], Training Loss: 0.9236
Accuracy on attack test dataset: 30.53%
Accuracy on clean test dataset: 75.68%
Epoch [1/10] - Attack Test Accuracy: 30.53%, Clean Test Accuracy: 75.68%
Epoch [2/10], Training Loss: 0.1769
Accuracy on attack test dataset: 92.63%
Accuracy on clean test dataset: 75.47%
Epoch [2/10] - Attack Test Accuracy: 92.63%, Clean Test Accuracy: 75.47%
Epoch [3/10], Training Loss: 0.0669
Accuracy on attack test dataset: 95.79%
Accuracy on clean test dataset: 73.27%
Epoch [3/10] - Attack Test Accuracy: 95.79%, Clean Test Accuracy: 73.27%
Epoch [4/10], Training Loss: 0.0920
Accuracy on attack test dataset: 94.74%
Accuracy on clean test dataset: 75.16%
Epoch [4/10] - Attack Test Accuracy: 94.74%, Clean Test Accuracy: 75.16%
Epoch [5/10], Training Loss: 0.0913
Accuracy on attack test dataset: 89.47%
Accuracy on clean test dataset: 76.42%
Epoch [5/10] - Attack Test Accuracy: 89.47%, Clean Test Accuracy: 76.42%
Epoch [6/10], Training Loss: 0.0482
Accuracy on attack test 